In [1]:
pip install google-api-python-client pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from googleapiclient.discovery import build
import pandas as pd

# YouTube API credentials
API_KEY = "9f00f3ic0wg3rg"  # Replace with your API key
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# Search parameters
QUERY = "ELSA Speak: English Learning"  # Search term
MAX_RESULTS = 10  # Number of videos to retrieve

# Initialize YouTube API client
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

def get_video_ids(query, max_results):
    """
    Search YouTube videos based on a query and return their video IDs.

    Args:
        query (str): Search query for YouTube videos.
        max_results (int): Maximum number of videos to retrieve.

    Returns:
        List[str]: List of video IDs.
    """
    search_response = youtube.search().list(
        q=query,
        part="id,snippet",
        maxResults=max_results,
        type="video"
    ).execute()

    video_ids = [item["id"]["videoId"] for item in search_response.get("items", [])]
    return video_ids

def get_comments(video_id):
    """
    Retrieve comments for a given video.

    Args:
        video_id (str): YouTube video ID.

    Returns:
        List[Dict]: List of comments and their metadata.
    """
    comments = []
    try:
        # Get comments from YouTube API
        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100
        ).execute()

        for item in response.get("items", []):
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "VideoID": video_id,
                "Author": comment["authorDisplayName"],
                "Comment": comment["textDisplay"],
                "Likes": comment["likeCount"],
                "PublishedAt": comment["publishedAt"]
            })

    except Exception as e:
        print(f"Error retrieving comments for video {video_id}: {e}")

    return comments

def main():
    # Step 1: Get video IDs related to the query
    video_ids = get_video_ids(QUERY, MAX_RESULTS)
    print(f"Found {len(video_ids)} videos for query: '{QUERY}'")

    # Step 2: Retrieve comments for each video
    all_comments = []
    for video_id in video_ids:
        print(f"Fetching comments for video: {video_id}")
        comments = get_comments(video_id)
        all_comments.extend(comments)

    # Step 3: Save comments to a CSV file
    if all_comments:
        df = pd.DataFrame(all_comments)
        df.to_csv("youtube_comments.csv", index=False)
        print("Comments saved to 'youtube_comments.csv'")
        print(df.head())
    else:
        print("No comments found.")

if __name__ == "__main__":
    main()
